In [ ]:
using Pkg
pkg"activate ."

In [ ]:
using AtlasRobot
using MeshCatMechanisms
using MeshCat
using Blink
using Rotations
using CoordinateTransformations
using RigidBodyDynamics
using RigidBodyDynamics.Graphs

In [ ]:
vis = Visualizer()
open(vis, Window())

In [ ]:
delete!(vis)
mechanism = AtlasRobot.mechanism()
mvis = MechanismVisualizer(mechanism, 
    URDFVisuals("/home/rdeits/.julia/dev/AtlasRobot/deps/Atlas/atlas.urdf", 
        package_path=[AtlasRobot.packagepath()]),
    vis)

In [ ]:
hand_urdf = "/home/rdeits/locomotion/drake-agile/drake/examples/Atlas/urdf/robotiq_simple.urdf"
hand = parse_urdf(Float64, hand_urdf)

for side in ["left", "right"]
    path_to_hand = [
        :world, 
        [Symbol(target(j, mechanism.tree)) for j in path(mechanism, root_body(mechanism), 
                                                         findbody(mechanism, "$(side[1])_hand"))]...,
        Symbol("after_$(side)_palm_joint")
    ]
    delete!(vis[path_to_hand...])
    MechanismVisualizer(hand, 
        URDFVisuals(hand_urdf, package_path=["/home/rdeits/locomotion/drake-agile/drake/examples"]),
        vis[path_to_hand...]
    )
    settransform!(vis[path_to_hand...], LinearMap(RotZ(π)) ∘ Translation(0, 0.15, 0))
end

In [ ]:
state = MechanismState(mechanism)
AtlasRobot.setnominal!(state)

set_configuration!(state, findjoint(mechanism, "r_arm_shz"), -0.8)
set_configuration!(state, findjoint(mechanism, "r_arm_shx"), 1.0)
set_configuration!(state, findjoint(mechanism, "r_arm_ely"), 2.5)
set_configuration!(state, findjoint(mechanism, "r_arm_elx"), -0.8)
set_configuration!(state, findjoint(mechanism, "r_arm_uwy"), 3)
set_configuration!(state, findjoint(mechanism, "l_arm_shz"), 0.8)
set_configuration!(state, findjoint(mechanism, "l_arm_shx"), -1.0)
set_configuration!(state, findjoint(mechanism, "l_arm_ely"), 2.5)
set_configuration!(state, findjoint(mechanism, "l_arm_elx"), 0.8)
set_configuration!(state, findjoint(mechanism, "l_arm_uwy"), 0)
copyto!(mvis, state)